# Export monthly temperatures
- This script is used to export temperatures from simulations;
- Simulations: CNTL, TRAF at FR-Capitole. 

In [1]:
import os
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
home_path = '/gws/nopw/j04/duicv/yuansun/'

In [2]:
archive = f'{home_path}0_urban_traffic/archive'
start_date = '2012-01-01T00:00:00'
end_date = '2012-12-31T23:00:00'
case_list = ['cntl', 'traffic'] 
GRIDNAME_single = 'FR-Cap'  

In [3]:
var_list_hourly = ['TSA_U', 'TBUILD', 'TG_U']
df_hourly = pd.DataFrame()
for case in case_list:
    ds_case = xr.open_dataset(f'{archive}/{GRIDNAME_single}_{case}/lnd/hist/{GRIDNAME_single}_{case}.clm2.h0.2004-02-20-03600.nc')
    for var in var_list_hourly : 
        ds_case_var = ds_case[var] - 273.15
        df_case_var=ds_case_var.to_dataframe().reset_index()[['time', var]]
        df_case_var.rename(columns={var: f'{var}_{case}'}, inplace=True)
        df_case_var['time'] = df_case_var['time'].dt.round('min').dt.ceil('min')
        df_case_var['time'] = pd.to_datetime(df_case_var['time'])
        if df_hourly.empty:
            df_hourly = df_case_var
        else:
            df_hourly = pd.merge(df_hourly, df_case_var, how='outer', on='time') 
df_hourly['time'] = df_hourly['time'] - pd.Timedelta(hours=1)
df_hourly['outdoor_indoor_cntl'] = df_hourly['TSA_U_cntl'] - df_hourly['TBUILD_cntl']
df_hourly['outdoor_indoor_traffic'] = df_hourly['TSA_U_traffic'] - df_hourly['TBUILD_traffic']
df_hourly['date'] = df_hourly['time'].dt.date
df_hourly2daily = df_hourly.groupby(['date']).mean().reset_index()          
df_hourly2daily.drop(columns=['time'], inplace=True)
df_hourly2daily['month'] = pd.to_datetime(df_hourly2daily['date']).dt.month  
df_hourly2daily.drop(columns=['date'], inplace=True)
df_monthly = df_hourly2daily.groupby(['month']).mean().reset_index()
df_monthly_5 = df_hourly2daily.groupby(['month']).quantile(0.05).reset_index()
df_monthly_95 = df_hourly2daily.groupby(['month']).quantile(0.95).reset_index()
df_monthly.head()

,month,TSA_U_cntl,TBUILD_cntl,TG_U_cntl,TSA_U_traffic,TBUILD_traffic,TG_U_traffic,outdoor_indoor_cntl,outdoor_indoor_traffic
0,1,5.811174,11.950035,5.678090,6.378428,11.951697,6.430495,-6.138860,-5.573270
1,2,5.236279,11.950012,5.688601,5.634785,11.950636,6.294222,-6.713733,-6.315851
2,3,10.039447,13.134769,11.738712,10.392680,13.313963,12.308396,-3.095322,-2.921283
3,4,12.886469,14.445519,14.985774,13.193615,14.761338,15.511022,-1.559050,-1.567724
4,5,17.541105,19.132153,21.127003,17.817484,19.481813,21.644049,-1.591048,-1.664330


In [4]:
var_list_daily = ['TGMNAV_U', 'TGMXAV_U', 'TREFMXAV_U', 'TREFMNAV_U']
df_daily = pd.DataFrame()
for case in case_list:
    ds_case_daily = xr.open_dataset(f'{archive}/{GRIDNAME_single}_{case}/lnd/hist/{GRIDNAME_single}_{case}.clm2.h1.2004-02-21-00000.nc')
    for var in var_list_daily:
        ds_case_var = ds_case_daily[var] - 273.15
        df_case_var=ds_case_var.to_dataframe().reset_index()[['time', var]]
        df_case_var.rename(columns={var: f'{var}_{case}'}, inplace=True)
        df_case_var['time'] = df_case_var['time'].dt.round('min').dt.ceil('min')
        df_case_var['time'] = pd.to_datetime(df_case_var['time'])
        if df_daily.empty:
            df_daily = df_case_var
        else:
            df_daily = pd.merge(df_daily, df_case_var, how='outer', on='time') 
df_daily['month'] = df_daily['time'].dt.month 
df_daily['TG_daynight_cntl'] = df_daily['TGMXAV_U_cntl'] - df_daily['TGMNAV_U_cntl']
df_daily['TG_daynight_traffic'] = df_daily['TGMXAV_U_traffic'] - df_daily['TGMNAV_U_traffic']
df_daily['TSA_daynight_cntl'] = df_daily['TREFMXAV_U_cntl'] - df_daily['TREFMNAV_U_cntl']
df_daily['TSA_daynight_traffic'] = df_daily['TREFMXAV_U_traffic'] - df_daily['TREFMNAV_U_traffic']
df_daily_delta = df_daily.groupby(['month']).mean().reset_index()
df_daily_delta_5 = df_daily.groupby(['month']).quantile(0.05).reset_index()
df_daily_delta_95 = df_daily.groupby(['month']).quantile(0.95).reset_index()
df_daily_delta.drop(columns=['time'], inplace=True)
df_daily_delta_5.drop(columns=['time'], inplace=True)
df_daily_delta_95.drop(columns=['time'], inplace=True)
df_daily_delta.head()          

,month,TGMNAV_U_cntl,TGMXAV_U_cntl,TREFMXAV_U_cntl,TREFMNAV_U_cntl,TGMNAV_U_traffic,TGMXAV_U_traffic,TREFMXAV_U_traffic,TREFMNAV_U_traffic,TG_daynight_cntl,TG_daynight_traffic,TSA_daynight_cntl,TSA_daynight_traffic
0,1,1.933324,11.497117,9.294834,3.100197,2.655987,12.264533,9.713933,3.653710,9.563792,9.608546,6.194637,6.060223
1,2,1.497296,13.656822,9.507830,2.918279,2.045319,14.295203,9.835820,3.386805,12.159526,12.249885,6.589550,6.449015
2,3,4.567094,24.234447,15.230643,5.842275,5.038572,24.856493,15.529846,6.221642,19.667355,19.817921,9.388368,9.308204
3,4,7.432055,27.058971,17.778589,8.662198,7.865685,27.637142,18.037601,8.969970,19.626917,19.771458,9.116392,9.067631
4,5,10.976097,36.808754,23.606508,12.208489,11.359016,37.388889,23.848543,12.446651,25.832657,26.029871,11.398019,11.401893


In [5]:
df_all = pd.merge(df_monthly, df_daily_delta, how='outer', on='month')
df_all_5 = pd.merge(df_monthly_5, df_daily_delta_5, how='outer', on='month')
df_all_95 = pd.merge(df_monthly_95, df_daily_delta_95, how='outer', on='month')
df_all.to_csv('./data_for_figure/monthly_mean.csv', index=False)
df_all_5.to_csv('./data_for_figure/monthly_5.csv', index=False)
df_all_95.to_csv('./data_for_figure/monthly_95.csv', index=False)
df_all.head()

,month,TSA_U_cntl,TBUILD_cntl,TG_U_cntl,TSA_U_traffic,TBUILD_traffic,TG_U_traffic,outdoor_indoor_cntl,outdoor_indoor_traffic,TGMNAV_U_cntl,...,TREFMXAV_U_cntl,TREFMNAV_U_cntl,TGMNAV_U_traffic,TGMXAV_U_traffic,TREFMXAV_U_traffic,TREFMNAV_U_traffic,TG_daynight_cntl,TG_daynight_traffic,TSA_daynight_cntl,TSA_daynight_traffic
0,1,5.811174,11.950035,5.678090,6.378428,11.951697,6.430495,-6.138860,-5.573270,1.933324,...,9.294834,3.100197,2.655987,12.264533,9.713933,3.653710,9.563792,9.608546,6.194637,6.060223
1,2,5.236279,11.950012,5.688601,5.634785,11.950636,6.294222,-6.713733,-6.315851,1.497296,...,9.507830,2.918279,2.045319,14.295203,9.835820,3.386805,12.159526,12.249885,6.589550,6.449015
2,3,10.039447,13.134769,11.738712,10.392680,13.313963,12.308396,-3.095322,-2.921283,4.567094,...,15.230643,5.842275,5.038572,24.856493,15.529846,6.221642,19.667355,19.817921,9.388368,9.308204
3,4,12.886469,14.445519,14.985774,13.193615,14.761338,15.511022,-1.559050,-1.567724,7.432055,...,17.778589,8.662198,7.865685,27.637142,18.037601,8.969970,19.626917,19.771458,9.116392,9.067631
4,5,17.541105,19.132153,21.127003,17.817484,19.481813,21.644049,-1.591048,-1.664330,10.976097,...,23.606508,12.208489,11.359016,37.388889,23.848543,12.446651,25.832657,26.029871,11.398019,11.401893
